In [228]:
import pandas
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
import time
import re

# q = (pl.scan_csv(\"bundestagDE.csv\", has_header=True, ignore_errors=True, schema=schema)
#         # remove all chair speeches and speeches with words fewer than 11
#         .filter(pl.col(\"chair\")==False& (pl.col(\"terms\") > 10))
# 
#         # clean strings (remove interpellations, etc.)
#         .select(pl.col(\"text\").str.replace_all(\"\\(.*?\\)\",\"\").str.strip_chars())
#         .select(pl.col(\"text\").str.replace_all(\"-\",\"\").str.strip_chars())
#         .select(pl.col(\"text\").str.replace_all(\"_\",\"\").str.strip_chars())
#     
#         # splits strings at . and ? and !; explodes them to new rows 
#         .select(pl.col(\"text\").str.split(by=\".\",inclusive=True))
#         .explode(\"text\")
#         .select(pl.col(\"text\").str.split(by=\"!\",inclusive=True))
#         .explode(\"text\")
#         .select(pl.col(\"text\").str.split(by=\"?\",inclusive=True))
#         .explode(\"text\")
#         .select(pl.col(\"text\").str.replace_all(\"(^.*Präsiden.*$)\", \"\").str.strip_chars())
#         .select(pl.col(\"text\").str.replace_all(\"(^.*Damen und Herren.*$)\", \"\").str.strip_chars())
#         .select(pl.col(\"text\").str.replace_all(\"(^.*Vielen Dank.*$)\", \"\").str.strip_chars())
#         # remove all cleaned strings (empty strings)
#         .filter(pl.col(\"text\").map_elements(lambda x: len(x))>30)
#     )
#     df = q.collect()

def clean_xml_speech():
    pass

def get_parlamint_xml_speech(input_path, country_id):
    """"

    :param input_path:
    :param country_id: 
    :param dataframe: 
    """
    root = ET.parse(input_path).getroot()
    u_ids = []
    u_speeches = []
    person_ids = []
    for elem in root.findall("tei:text/tei:body/tei:div/tei:u", namespace):
        u_ids.append(elem.attrib[f"{{{namespace['xml']}}}id"])
        try:
            person_ids.append(f"{country_id}-{elem.attrib['who'][1:]}")
        except KeyError:
            person_ids.append("")
        u_speech = ""
        for seg in elem.findall("tei:seg", namespace):
            seg_text = seg.text
            # Not all segments end in a period. To ensure that splitting text by sentence goes well, and that sentences don't combine, I'm adding a period to the end of each segment.
            if seg_text[-1] not in [".", "?", "!"]:
                seg_text += "."
           # If there is no segment originally added to the total speech, add it with no starting space. If there is, then there will be a starting space (so the sentence is split).
            if not u_speech:
                u_speech += seg_text
            else:
                u_speech += f" {seg_text}"
        u_speeches.append(u_speech)
    return pd.DataFrame({"speech_id": u_ids, "speech": u_speeches, "person_id": person_ids})

# Parse the XML file
base_legislative_path = "ParliamentHackathon2024Data/ParlaMint4.0-GB/ParlaMint-GB.TEI/ParlaMint-taxonomy-parla.legislature.xml"
person_path = "ParliamentHackathon2024Data/ParlaMint4.0-GB/ParlaMint-GB.TEI/ParlaMint-GB-listPerson.xml"
org_path = "ParliamentHackathon2024Data/ParlaMint4.0-GB/ParlaMint-GB.TEI/ParlaMint-GB-listOrg.xml"
namespace = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace", "XInclude": "http://www.w3.org/2001/XInclude"}
        
corpus_list = ["ParlaMint-BA", "ParlaMint-BE", "ParlaMint-BG", "ParlaMint-CZ", "ParlaMint-DK", "ParlaMint-EE", "ParlaMint-ES", "ParlaMint-ES-CT", "ParlaMint-ES-GA", "ParlaMint-ES-PV", "ParlaMint-FI", "ParlaMint-FR", "ParlaMint-GR", "ParlaMint-HR", "ParlaMint-HU", "ParlaMint-IS", "ParlaMint-IT", "ParlaMint-LV", "ParlaMint-NL", "ParlaMint-NO", "ParlaMint-PL", "ParlaMint-PT", "ParlaMint-RS", "ParlaMint-SE", "ParlaMint-SI", "ParlaMint-TR", "ParlaMint-UA"]

corpus_list_test = ["ParlaMint-GB"]

#"ParlaMint-AT"
test_base_path = "D:\\ParlaMint Data\\Files"
for corpus in corpus_list_test:
    xml_file_paths = []
    parlamint_speeches = pandas.DataFrame(columns = ["speech_id", "speech", "person_id"])
    base_xml_path = f"{test_base_path}\\{corpus}\\{corpus}.TEI"
    xml_info_path = Path(f"{base_xml_path}\\{corpus}.xml")
    root = ET.parse(xml_info_path).getroot()

    for elem in root.findall("XInclude:include", namespace):
        xml_file_paths.append(Path(f"{base_xml_path}\\{elem.attrib["href"]}"))
    
    number_xmls = len(xml_file_paths)
    print(f"Beginning {corpus} search with {number_xmls} files.")
    for i, file_path in enumerate(xml_file_paths):
        new_speech = get_parlamint_xml_speech(file_path, corpus[-2:])
        # display(parlamint_speeches)
        # display(new_speech)
        # print(parlamint_speeches)
        parlamint_speeches = pd.concat([parlamint_speeches, new_speech])
        # print(f"Finished search {i+1} out of {number_xmls}.")
    
    #parlamint_speeches.to_csv(f"data-wrangling/files/{corpus}-RawSpeeches.csv", index=False)
    parlamint_speeches.to_csv(f"D:\\ParlaMint Data\\Raw Data\\{corpus}-RawSpeechesCSV.csv", index=False)
    parlamint_speeches.to_parquet(f"D:\\ParlaMint Data\\Raw Data\\{corpus}-RawSpeechesParquet.gzip", index=False,
              compression='gzip')





#get_legislature_taxonomy(base_legislative_path, "data-wrangling/files/legislature_taxonomy.csv")

# stuff = ["D:\ParlaMint Data\Files"]
# def get_xml_paths(input_paths, export_path):
#     root = ET.parse(input_path).getroot()
#     for elem in root.findall(".//tei:category[@w3:id='parla.organization']", namespace):
#         categories = [
#             [category.attrib[f"{{{namespace['w3']}}}id"], category.find("tei:catDesc/tei:term", namespace).text] for category in elem.findall("tei:category/tei:category", namespace) + elem.findall("tei:category/tei:category/tei:category", namespace)
#         ]
#         legislature_taxonomy = pandas.DataFrame(categories, columns = ["parla_tag", "house_type"])
#         legislature_taxonomy.to_csv(export_path, index=False)

Beginning ParlaMint-GB search with 2209 files.


In [229]:
new_dataframe = pd.read_parquet("D:\\ParlaMint Data\\Raw Data\\ParlaMint-GB-RawSpeechesParquet.gzip")
new_dataframe


,speech_id,speech,person_id
0,ParlaMint-GB_2015-01-05-commons.u1,1. What progress her Department has made on im...,GB-JenniferWillott
1,ParlaMint-GB_2015-01-05-commons.u2,The Government are on track to deliver their c...,GB-TheresaMay
2,ParlaMint-GB_2015-01-05-commons.u3,"It is clear that exit checks, which were scrap...",GB-JenniferWillott
3,ParlaMint-GB_2015-01-05-commons.u4,"As I indicated in my original answer, we are o...",GB-TheresaMay
4,ParlaMint-GB_2015-01-05-commons.u5,19. Given the situation at our border in Calai...,GB-AlexanderCunningham
...,...,...,...
670907,ParlaMint-GB_2022-07-21-lords.u209,I will have to check that point for the noble ...,GB-JoannaPenn
670908,ParlaMint-GB_2022-07-21-lords.u210,"My Lords, the Minister has referred a number o...",GB-PremSikka
670909,ParlaMint-GB_2022-07-21-lords.u211,The noble Lord is right that different aspects...,GB-JoannaPenn
670910,ParlaMint-GB_2022-07-21-lords.u212,I thank noble Lords for a very interesting deb...,GB-PremSikka


In [ ]:
gb_data = new_dataframe.copy()

def clean_text(text):
    text = re.sub(r'\(.*?\)', '', text)  # Remove interpellations (text within parentheses)
    text = re.sub(r'-', '', text)  # Remove hyphens
    text = re.sub(r'_', '', text)  # Remove underscores
    return text.strip()

def split_text(text):
    return re.split(r'(?<!\d)[.!?]\s+', text)

gb_data['speech'] = gb_data['speech'].apply(clean_text)

gb_data = gb_data.assign(speech=gb_data['speech'].apply(split_text)).explode('speech')

gb_data = gb_data[gb_data['speech'].str.len() > 30]

gb_data


In [55]:

# Getting the Party Information
def find_party():
    tree = ET.parse(base_legislative_path)
    root = tree.getroot()
    for elem in root:
        print(elem.tag, elem.attrib)
        
        

results = {}
tree = ET.parse(person_path)
root = tree.getroot()
# Setting up TEI namespace

#"{http://www.tei-c.org/ns/1.0}person"
for elem in root.findall("tei:person", namespace):
    # Getting name_ids
    name_id = elem.attrib[f"{{{namespace['xml']}}}id"]
    print(name_id)
    
    # Getting gender
    gender = [gen.attrib["value"] for gen in elem.findall("tei:sex", namespace)][0]    
    
    # Getting affiliations (has party, member role, year)
    affiliations = [affil.attrib for affil in elem.findall("tei:affiliation", namespace)]
    roles = set()
    parties = set()
    government = set()
    for affiliation in affiliations:
        if affiliation == affiliations[0]:
            term_start = affiliation["from"]
        try:
            if "party" in affiliation["ref"]:
                parties.add(affiliation["ref"])
            else:
                government.add(affiliation["ref"])
        except KeyError as e:
            pass
        roles.add(affiliation["role"])

    # print(roles)
    # print(parties)
    # print(government)
    # print(term_start)
    
    # Getting the full name of the individual
    forename = ' '.join([name.text for name in elem.findall('tei:persName/tei:forename', namespace)])
    surname = ' '.join([name.text for name in elem.findall('tei:persName/tei:surname', namespace)])
    name = f"{forename}, {surname}"
    
    # Updating the results dictionary
    results.update({"name_id": name_id, "name": name, "gender": gender, "term_start": term_start, "roles": roles, "parties": parties, "government": government, "affiliations": affiliations})
    
    #print("exception")
    #print(results)
    # print(affiliations)
    # print(name)
    # for person 
    # 
    # in list(elem):
    #     # print(person.tag, person.attrib)
    #     #print(person.tag, person.attrib)
    # #print(elem.tag, elem.attrib)
    #     if elem.tag == "{http://www.tei-c.org/ns/1.0}head":
    #         # The header contains the debate topic. Attribute notes which header it is.
    #         #print(elem.attrib)
    #         debate_topic = elem.text.strip()
    #         print(elem.text.strip())
    #     elif elem.tag == "{http://www.tei-c.org/ns/1.0}u":
    #         #print(elem.attrib)
    #         pass
    #     #print(elem.tag)
    #     try:
    #         pass
    #         #print(elem.attrib["{http://www.w3.org/XML/1998/namespace}id"])
    #     except Exception as e:
    #         pass
    #     # print(elem.tag, "hola",elem.attrib)
    #     try:
    #         if elem.text.strip():
    #             pass
    #             # print(elem.tag)
    #             # print(elem.attrib)
    #             # print(elem.text.strip())
    #             #results[elem.tag] = elem.text.strip()
    #     except Exception as e:
    #         pass
    #         #print(e)

MargaretProsser
JonathanMendelsohn
AlanMak
RuthHunt
JoannaCherry
WilliamJordan
MayBlood
TheresaVilliers
GeorgeGalloway
PaulWhite
JohnSpellar
BeebanKidron
AnnaTurley
GeraintDavies
TimothyYeo
PeterHeatonJones
GrevilleHoward
NicholasHoltam
DavidLaws
JohnGardiner
EleanorReeves
RobertMay
EmmaLewellBuck
GillianKeegan
MatthewWestern
MargotAline
SebastianCoe
BrendaDean
SueEllenBraverman
KatyClark
RobertFlello
GavinShuker
StewartJackson
MarkEastwood
RogerGale
MelanieOnn
AngelaHarris
DonaldMackay
CharlesDugdale
JulianSmith
JamesGray
ThomasBrake
ReginaldEmpey
GarryHart
TaniaMathias
LisaForbes
HilaryBenn
LaurenceRobertson
JeanBarker
ThomasCoke
DianaEccles
JaneHunt
MarkHunter
RobertSyms
TobiasEllwood
LeoDocherty
SallyAnnHart
TomGreatrex
AndrewGriffiths
ErnestOxburgh
BrooksNewmark
AlanBeith
DavidRichards
VictoriaAtkins
AnnMcKechin
RobertMaclennan
CharlesHendry
BrettElphicke
MiriamCates
TracyBrabin
MichaelMorris
PaulCondon
SallyOppenheimBarnes
PaulMurphy
ThangamDebbonaire
GavinBarwell
EwenCameron
Pat

In [7]:
find_party()

{http://www.tei-c.org/ns/1.0}desc {'{http://www.w3.org/XML/1998/namespace}lang': 'en'}
{http://www.tei-c.org/ns/1.0}category {'{http://www.w3.org/XML/1998/namespace}id': 'parla.geo-political'}
{http://www.tei-c.org/ns/1.0}category {'{http://www.w3.org/XML/1998/namespace}id': 'parla.organization'}
{http://www.tei-c.org/ns/1.0}category {'{http://www.w3.org/XML/1998/namespace}id': 'parla.term'}
